In [ ]:
# This notebook extract the names for all images that has row scores and sort them
# also, the names can be used to find the files that will be used in feature extraction.
# the user info can be used as 

In [71]:
import pandas as pd
import numpy as np
from collections import defaultdict 
import json, os
import shutil
from tqdm.notebook import tqdm

In [44]:
input_file = '2022年度_修論引継資料_下村隼生/子育て世帯向け住宅の需要側分析/アンケート調査_回答データ/本調査/内観_間取り/20221208image_report_raw_all.tsv'
df_rawscore = pd.read_csv(input_file, sep='\t')
df_rawscore = df_rawscore[df_rawscore.imageFileA.str.match(r'[A-F]_')]
df_rawscore.reset_index()

,index,imageFileA,imageFileB,segmentGender,segmentGeneration,userIdentifierCode,userGender,userAge,userPrefecture,userAddress,Q1,Q2,Q3,Q4,Q5,thinkingTime,AB
0,0,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,40,1ec61dec451c2384086fc4700c98961473c9b9c2,1,47,徳島県,鳴門市,0.0,0.0,0.0,0.0,0.0,43,AB
1,1,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,30,631126bde85a19caa08198a45cfcf7e091435293,1,37,静岡県,掛川市,-0.5,-0.5,0.5,0.5,0.5,20,BA
2,2,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,264ba091da1a13cb7a539259e79e8ee98793a2bb,1,44,東京都,大田区,1.0,1.0,1.0,1.0,1.0,142,AB
3,3,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,2911ac36565c625a777c4c074381628bfbee2ed0,1,42,東京都,福生市,-0.5,-0.5,0.0,0.5,-0.5,24,BA
4,4,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_105_0.png,A_105_1.jpg,A_105_2.jpg,A_105_3.jp...",1,40,27e52c843455e55e5d4d08af26ca47e004a73103,1,43,京都府,宇治市,0.5,0.5,0.5,0.5,0.0,39,AB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,27375,"A_127_0.png,A_127_1.jpg,A_127_2.jpg,A_127_3.jp...","A_35_0.gif,A_35_1.jpg,A_35_2.jpg,A_35_3.jpg,A_...",2,30,fd155c5760b04a991e7451ebc8b03b6ef19122b8,2,31,埼玉県,朝霞市,-0.5,-0.5,-0.5,-0.5,-0.5,31,BA
3796,27376,"A_88_0.png,A_88_1.jpg,A_88_2.jpg,A_88_3.jpg,A_...","C_36_0.jpg,C_36_1.jpg,C_36_2.jpg,C_36_3.jpg,C_...",2,20,30f25081de049de3d2c0e66ea664b51cde1c4ecc,2,28,鳥取県,米子市,0.0,-0.5,-0.5,-0.5,-0.5,56,AB
3797,27377,"D_26_0.jpg,D_26_1.jpg,D_26_2.jpg,D_26_3.jpg,D_...","A_21_0.png,A_21_1.jpg,A_21_2.jpg,A_21_3.jpg,A_...",2,30,0be141f1997339e69d766823ab2c5de01548c34d,2,30,北海道,札幌市厚別区,-1.0,-1.0,-1.0,-1.0,-1.0,34,BA
3798,27378,"D_60_0.jpg,D_60_1.jpg,D_60_2.jpg,D_60_3.jpg,D_...","D_27_0.jpg,D_27_1.jpg,D_27_2.jpg,D_27_3.jpg,D_...",2,60,dd859ae4a3ac5064e26f135871593b0a4712c778,2,60,神奈川県,川崎市宮前区,-0.5,0.5,-0.5,-0.5,-0.5,24,BA


In [45]:
def extract_image_filename(images:list):
    filename = '_'.join(images.split(',')[0].split('_')[:2])
    return filename

In [46]:
df_rawscore['Afilename'] = df_rawscore.imageFileA.apply(extract_image_filename)
df_rawscore['Bfilename'] = df_rawscore.imageFileB.apply(extract_image_filename)
df_rawscore

,imageFileA,imageFileB,segmentGender,segmentGeneration,userIdentifierCode,userGender,userAge,userPrefecture,userAddress,Q1,Q2,Q3,Q4,Q5,thinkingTime,AB,Afilename,Bfilename
0,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,40,1ec61dec451c2384086fc4700c98961473c9b9c2,1,47,徳島県,鳴門市,0.0,0.0,0.0,0.0,0.0,43,AB,F_43,A_126
1,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,30,631126bde85a19caa08198a45cfcf7e091435293,1,37,静岡県,掛川市,-0.5,-0.5,0.5,0.5,0.5,20,BA,F_43,A_126
2,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,264ba091da1a13cb7a539259e79e8ee98793a2bb,1,44,東京都,大田区,1.0,1.0,1.0,1.0,1.0,142,AB,D_50,D_59
3,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,2911ac36565c625a777c4c074381628bfbee2ed0,1,42,東京都,福生市,-0.5,-0.5,0.0,0.5,-0.5,24,BA,D_50,D_59
4,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_105_0.png,A_105_1.jpg,A_105_2.jpg,A_105_3.jp...",1,40,27e52c843455e55e5d4d08af26ca47e004a73103,1,43,京都府,宇治市,0.5,0.5,0.5,0.5,0.0,39,AB,F_43,A_105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27375,"A_127_0.png,A_127_1.jpg,A_127_2.jpg,A_127_3.jp...","A_35_0.gif,A_35_1.jpg,A_35_2.jpg,A_35_3.jpg,A_...",2,30,fd155c5760b04a991e7451ebc8b03b6ef19122b8,2,31,埼玉県,朝霞市,-0.5,-0.5,-0.5,-0.5,-0.5,31,BA,A_127,A_35
27376,"A_88_0.png,A_88_1.jpg,A_88_2.jpg,A_88_3.jpg,A_...","C_36_0.jpg,C_36_1.jpg,C_36_2.jpg,C_36_3.jpg,C_...",2,20,30f25081de049de3d2c0e66ea664b51cde1c4ecc,2,28,鳥取県,米子市,0.0,-0.5,-0.5,-0.5,-0.5,56,AB,A_88,C_36
27377,"D_26_0.jpg,D_26_1.jpg,D_26_2.jpg,D_26_3.jpg,D_...","A_21_0.png,A_21_1.jpg,A_21_2.jpg,A_21_3.jpg,A_...",2,30,0be141f1997339e69d766823ab2c5de01548c34d,2,30,北海道,札幌市厚別区,-1.0,-1.0,-1.0,-1.0,-1.0,34,BA,D_26,A_21
27378,"D_60_0.jpg,D_60_1.jpg,D_60_2.jpg,D_60_3.jpg,D_...","D_27_0.jpg,D_27_1.jpg,D_27_2.jpg,D_27_3.jpg,D_...",2,60,dd859ae4a3ac5064e26f135871593b0a4712c778,2,60,神奈川県,川崎市宮前区,-0.5,0.5,-0.5,-0.5,-0.5,24,BA,D_60,D_27


In [89]:
# this part is optional if we go for pair-wise training
image_score_dct = defaultdict(int)
image_count_dct = defaultdict(int)
for index, row in df_rawscore.iterrows():
    image_count_dct[row['Afilename']] += 1
    image_count_dct[row['Bfilename']] += 1
    if row['Q1'] > 0:
        # B is better
        image_score_dct[row['Bfilename']] += 1
    if row['Q1'] < 0:
        # A is better
        image_score_dct[row['Afilename']] += 1
# print(image_score_dct)
# print(image_count_dct)

# calculate avg score
for key in image_score_dct.keys():
    image_score_dct[key] = image_score_dct[key] / image_count_dct[key]
image_count_dct  
image_score_dct = sorted(image_score_dct.items(), key=lambda item: item[1], reverse=True)
image_score_dct = {image:f'{score:.4f}' for image, score in image_score_dct}
image_count_dct = sorted(image_count_dct.items(), key=lambda item: item[1], reverse=True)
print(image_score_dct)
print(image_count_dct)
with open('image_score.json', 'w') as file:
    json.dump(image_score_dct, file)

{'D_64': '0.6786', 'F_43': '0.6480', 'A_35': '0.5650', 'D_24': '0.5583', 'D_38': '0.5300', 'A_91': '0.5111', 'A_127': '0.5040', 'F_51': '0.4913', 'A_126': '0.4905', 'A_124': '0.4842', 'D_51': '0.4789', 'A_105': '0.4783', 'A_94': '0.4706', 'F_42': '0.4579', 'A_95': '0.4304', 'D_59': '0.4227', 'D_37': '0.4188', 'C_26': '0.4160', 'D_30': '0.4136', 'D_26': '0.4091', 'D_19': '0.4000', 'D_42': '0.3952', 'D_65': '0.3947', 'D_40': '0.3913', 'A_21': '0.3909', 'D_45': '0.3786', 'D_29': '0.3714', 'D_27': '0.3647', 'D_60': '0.3476', 'D_50': '0.3381', 'A_88': '0.3286', 'D_25': '0.3067', 'C_36': '0.2957', 'D_34': '0.2938', 'D_28': '0.2786', 'D_46': '0.2632', 'F_13': '0.2000', 'D_53': '0.1714'}
[('F_13', 260), ('F_43', 250), ('C_26', 250), ('A_127', 250), ('A_105', 230), ('D_40', 230), ('C_36', 230), ('F_51', 230), ('A_95', 230), ('D_59', 220), ('A_21', 220), ('D_26', 220), ('D_30', 220), ('A_126', 210), ('D_50', 210), ('D_53', 210), ('D_60', 210), ('D_42', 210), ('D_29', 210), ('D_19', 210), ('A_88'

In [48]:
user_features = df_rawscore[['segmentGender', 'segmentGeneration', 'userAge', 'thinkingTime', 'userPrefecture', 'userAddress']]
user_features

,segmentGender,segmentGeneration,userAge,thinkingTime,userPrefecture,userAddress
0,1,40,47,43,徳島県,鳴門市
1,1,30,37,20,静岡県,掛川市
2,1,40,44,142,東京都,大田区
3,1,40,42,24,東京都,福生市
4,1,40,43,39,京都府,宇治市
...,...,...,...,...,...,...
27375,2,30,31,31,埼玉県,朝霞市
27376,2,20,28,56,鳥取県,米子市
27377,2,30,30,34,北海道,札幌市厚別区
27378,2,60,60,24,神奈川県,川崎市宮前区


In [53]:
# Converting 'userPrefecture' and 'userAddress' columns into indices

# Extract unique values from both columns and sort them
unique_prefectures = df_rawscore['userPrefecture'].unique()
unique_addresses = df_rawscore['userAddress'].unique()

# Create dictionaries to map prefectures and addresses to indices
prefecture_to_index = {prefecture: index for index, prefecture in enumerate(unique_prefectures)}
address_to_index = {address: index for index, address in enumerate(unique_addresses)}

# Write the dictionary to a file in JSON format
with open('prefecture_to_index.json', 'w') as file:
    json.dump(prefecture_to_index, file)
with open('address_to_index.json', 'w') as file:
    json.dump(prefecture_to_index, file)

# convert names into index
df_rawscore['userPrefectureIds'] = df_rawscore.userPrefecture.apply(lambda x:prefecture_to_index[x])
df_rawscore['userAddressIds'] = df_rawscore.userAddress.apply(lambda x:address_to_index[x])


In [51]:
# convert user age into range indices
df_rawscore['userAgeIds'] = df_rawscore.userAge.apply(lambda x: x // 5)

In [93]:
df_rawscore.to_csv('samples/demo_features/rawscore.csv')
df_rawscore

,imageFileA,imageFileB,segmentGender,segmentGeneration,userIdentifierCode,userGender,userAge,userPrefecture,userAddress,Q1,...,Q3,Q4,Q5,thinkingTime,AB,Afilename,Bfilename,userAgeIds,userPrefectureIds,userAddressIds
0,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,40,1ec61dec451c2384086fc4700c98961473c9b9c2,1,47,徳島県,鳴門市,0.0,...,0.0,0.0,0.0,43,AB,F_43,A_126,9,0,0
1,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_126_0.png,A_126_1.jpg,A_126_2.jpg,A_126_3.jp...",1,30,631126bde85a19caa08198a45cfcf7e091435293,1,37,静岡県,掛川市,-0.5,...,0.5,0.5,0.5,20,BA,F_43,A_126,7,1,1
2,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,264ba091da1a13cb7a539259e79e8ee98793a2bb,1,44,東京都,大田区,1.0,...,1.0,1.0,1.0,142,AB,D_50,D_59,8,2,2
3,"D_50_0.jpg,D_50_1.jpg,D_50_2.jpg,D_50_3.jpg,D_...","D_59_0.jpg,D_59_1.jpg,D_59_2.jpg,D_59_3.jpg,D_...",1,40,2911ac36565c625a777c4c074381628bfbee2ed0,1,42,東京都,福生市,-0.5,...,0.0,0.5,-0.5,24,BA,D_50,D_59,8,2,3
4,"F_43_0.jpg,F_43_1.jpg,F_43_2.jpg,F_43_3.jpg,F_...","A_105_0.png,A_105_1.jpg,A_105_2.jpg,A_105_3.jp...",1,40,27e52c843455e55e5d4d08af26ca47e004a73103,1,43,京都府,宇治市,0.5,...,0.5,0.5,0.0,39,AB,F_43,A_105,8,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27375,"A_127_0.png,A_127_1.jpg,A_127_2.jpg,A_127_3.jp...","A_35_0.gif,A_35_1.jpg,A_35_2.jpg,A_35_3.jpg,A_...",2,30,fd155c5760b04a991e7451ebc8b03b6ef19122b8,2,31,埼玉県,朝霞市,-0.5,...,-0.5,-0.5,-0.5,31,BA,A_127,A_35,6,19,583
27376,"A_88_0.png,A_88_1.jpg,A_88_2.jpg,A_88_3.jpg,A_...","C_36_0.jpg,C_36_1.jpg,C_36_2.jpg,C_36_3.jpg,C_...",2,20,30f25081de049de3d2c0e66ea664b51cde1c4ecc,2,28,鳥取県,米子市,0.0,...,-0.5,-0.5,-0.5,56,AB,A_88,C_36,5,47,512
27377,"D_26_0.jpg,D_26_1.jpg,D_26_2.jpg,D_26_3.jpg,D_...","A_21_0.png,A_21_1.jpg,A_21_2.jpg,A_21_3.jpg,A_...",2,30,0be141f1997339e69d766823ab2c5de01548c34d,2,30,北海道,札幌市厚別区,-1.0,...,-1.0,-1.0,-1.0,34,BA,D_26,A_21,6,13,522
27378,"D_60_0.jpg,D_60_1.jpg,D_60_2.jpg,D_60_3.jpg,D_...","D_27_0.jpg,D_27_1.jpg,D_27_2.jpg,D_27_3.jpg,D_...",2,60,dd859ae4a3ac5064e26f135871593b0a4712c778,2,60,神奈川県,川崎市宮前区,-0.5,...,-0.5,-0.5,-0.5,24,BA,D_60,D_27,12,28,410


In [92]:
df_features = df_rawscore[['Afilename', 'Bfilename', 'userGender', 'userAgeIds', 'userPrefectureIds', 'userAddressIds', 'Q1']]
df_features = df_features.reset_index(drop=True)
df_features.to_csv('samples/demo_features/user_features.csv')
df_features

,Afilename,Bfilename,userGender,userAgeIds,userPrefectureIds,userAddressIds,Q1
0,F_43,A_126,1,9,0,0,0.0
1,F_43,A_126,1,7,1,1,-0.5
2,D_50,D_59,1,8,2,2,1.0
3,D_50,D_59,1,8,2,3,-0.5
4,F_43,A_105,1,8,3,4,0.5
...,...,...,...,...,...,...,...
3795,A_127,A_35,2,6,19,583,-0.5
3796,A_88,C_36,2,5,47,512,0.0
3797,D_26,A_21,2,6,13,522,-1.0
3798,D_60,D_27,2,12,28,410,-0.5


In [90]:
# note: for perfectures and address, real index should cover all the possible locations which means using post code is better.

In [76]:
# copy all the files into a different location for easier access in future.
images_root = '/home/wankun/yuan/2022年度_修論引継資料_下村隼生/子育て世帯向け住宅の供給側分析/内観特徴分析/scraping_images/'
subfolder_dict = {'A':'A_大和ハウス工業', 'C':'C_積水化学工業', 'D':'D_トヨタホーム', 'F':'F_パナソニックホームズ' }
def copy_images(filenames, output_dir, missing_dict):
    # Copy the file
    key = filenames[0] # A- F
    filename_lst = filenames.split(',')
    folder = subfolder_dict[key]
    for file in filename_lst:
        name, extension = os.path.splitext(file)
        source = os.path.join(images_root, folder, '内外観画像_child', f'{name}.jpg')
        if not os.path.isfile(source):
            missing_dict[source] += 1
            if missing_dict[source] == 1:
                print('Warning: image not found -->', source)      
        elif not os.path.isfile(os.path.join(output_dir, f'{name}.jpg')):
            shutil.copy(source, output_dir)

missing_dict = defaultdict(int)
for index, row in tqdm(df_rawscore.iterrows()):
    A_target = f'samples/demo_features/{row["Afilename"]}'
    B_target = f'samples/demo_features/{row["Bfilename"]}'
    os.makedirs(A_target, exist_ok=True)
    os.makedirs(B_target, exist_ok=True)
    # Check if the destination file exists
    copy_images(row['imageFileA'], A_target, missing_dict)
    copy_images(row['imageFileB'], B_target, missing_dict)
print(missing_dict)

0it [00:00, ?it/s]

defaultdict(<class 'int'>, {'/home/wankun/yuan/2022年度_修論引継資料_下村隼生/子育て世帯向け住宅の供給側分析/内観特徴分析/scraping_images/A_大和ハウス工業/内外観画像_child/A_35_0.jpg': 200})
